In [256]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('original_dataset.csv', low_memory = False, dtype = {'locality': str, 'postcode': str, 'property_subtype': str})
raw_df = df

In [257]:
def extract_address(d): 
    return re.sub("([A-Za-z0-9àâçéèêëîïôûùüÿñæœ .-]*)(,{0,} {0,})(\d{4})([A-Za-zàâçéèêëîïôûùüÿñæœ .-]*)(,{0,} {0,})([A-Za-zàâçéèêëîïôûùüÿñæœ .-]*)", r'\1', d)
def extract_postcode(d):
    return re.sub("([A-Za-z0-9àâçéèêëîïôûùüÿñæœ .-]*)(,{0,} {0,})(\d{4})([A-Za-zàâçéèêëîïôûùüÿñæœ .-]*)(,{0,} {0,})([A-Za-zàâçéèêëîïôûùüÿñæœ .-]*)", r'\3', d)
def extract_locality(d): 
    return re.sub("([A-Za-z0-9àâçéèêëîïôûùüÿñæœ .-]*)(,{0,} {0,})(\d{4})([A-Za-zàâçéèêëîïôûùüÿñæœ .-]*)(,{0,} {0,})([A-Za-zàâçéèêëîïôûùüÿñæœ .-]*)", r'\4', d)

In [258]:
df['extracted_address'] = df[~df['locality'].isnull()]['locality'].apply(extract_address)
df['extracted_postcode'] = df[~df['locality'].isnull()]['locality'].apply(extract_postcode)
df['extracted_locality'] = df[~df['locality'].isnull()]['locality'].apply(extract_locality)


In [259]:
import numpy as np
df['merged_postcode'] = df['postcode'].replace(np.nan, '') + df['extracted_postcode'].replace(np.nan, '')

In [260]:
df.head()

,source,hyperlink,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,...,garden_area,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state,extracted_address,extracted_postcode,extracted_locality,merged_postcode
0,6,8901695,4180,4180,True,MIXED_USE_BUILDING,295000,NaN,3,242,...,1000,1403,1403,0,False,GOOD,,4180,,41804180
1,6,8747010,8730,8730,True,VILLA,675000,NaN,4,349,...,977,1526,1526,0,False,AS_NEW,,8730,,87308730
2,6,8775843,4020,4020,True,APARTMENT_BLOCK,250000,NaN,5,303,...,0,760,760,0,False,TO_RENOVATE,,4020,,40204020
3,6,8910441,1200,1200,True,HOUSE,545000,NaN,4,235,...,0,63,63,0,False,JUST_RENOVATED,,1200,,12001200
4,6,8758672,1190,1190,True,MIXED_USE_BUILDING,500000,NaN,2,220,...,60,193,193,0,False,AS_NEW,,1190,,11901190


In [261]:
def unmerge_postcode(d): 
    return re.sub("(\d{4}){0,1}(\d{4})", r'\1', d)

In [262]:
df['unmerged_postcode'] = df[~df['merged_postcode'].isnull()]['merged_postcode'].apply(unmerge_postcode)
df.head()

,source,hyperlink,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,...,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state,extracted_address,extracted_postcode,extracted_locality,merged_postcode,unmerged_postcode
0,6,8901695,4180,4180,True,MIXED_USE_BUILDING,295000,NaN,3,242,...,1403,1403,0,False,GOOD,,4180,,41804180,4180
1,6,8747010,8730,8730,True,VILLA,675000,NaN,4,349,...,1526,1526,0,False,AS_NEW,,8730,,87308730,8730
2,6,8775843,4020,4020,True,APARTMENT_BLOCK,250000,NaN,5,303,...,760,760,0,False,TO_RENOVATE,,4020,,40204020,4020
3,6,8910441,1200,1200,True,HOUSE,545000,NaN,4,235,...,63,63,0,False,JUST_RENOVATED,,1200,,12001200,1200
4,6,8758672,1190,1190,True,MIXED_USE_BUILDING,500000,NaN,2,220,...,193,193,0,False,AS_NEW,,1190,,11901190,1190


In [263]:
df[df['merged_postcode'].astype(str).map(len) == 4]
#issue when one of both values is empty

,source,hyperlink,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,...,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state,extracted_address,extracted_postcode,extracted_locality,merged_postcode,unmerged_postcode
18034,2,https://www.realo.be/en/rue-rimiere-105-4120-r...,"Rue Rimière 105, 4120 Rotheux-Rimière, Neupré",NaN,True,House,0,NaN,0,0,...,NaN,No,NaN,False,NaN,Rue Rimière 105,4120,Rotheux-Rimière,4120,
18035,2,https://www.realo.be/en/sint-truidensesteenweg...,"Sint-Truidensesteenweg 95A A, 3350 Orsmaal-Gus...",NaN,True,House,340000€ 340.000,NaN,3,170m2,...,NaN,Yes,NaN,False,NaN,Sint-Truidensesteenweg 95A A,3350,Orsmaal-Gussenhoven,3350,
18036,2,https://www.realo.be/en/augustijnenlaan-77-220...,"Augustijnenlaan 77 201, 2200 Herentals",NaN,False,Flat,259000€ 259.000,NaN,2,88m2,...,NaN,Yes,NaN,False,NaN,Augustijnenlaan 77 201,2200,Herentals,2200,
18037,2,https://www.realo.be/en/averbodesesteenweg-19-...,"Averbodesesteenweg 19, 2230 Herselt",NaN,True,House,449000€ 449.000,NaN,4,303m2,...,NaN,Yes,NaN,False,NaN,Averbodesesteenweg 19,2230,Herselt,2230,
18038,2,https://www.realo.be/en/edward-pynaertkaai-21-...,"Edward pynaertkaai 21, 9050 Ledeberg",NaN,False,Flat,249900€ 249.900,NaN,3,102m2,...,NaN,Yes,NaN,False,NaN,Edward pynaertkaai 21,9050,Ledeberg,9050,
18040,2,https://www.realo.be/en/groenplein-33-9060-zel...,"Groenplein 33 302, 9060 Zelzate",NaN,False,Flat,159000€ 159.000,NaN,2,244m2,...,NaN,Yes,NaN,False,NaN,Groenplein 33 302,9060,Zelzate,9060,
18041,2,https://www.realo.be/en/hoogveldstraat-21-3720...,"Hoogveldstraat 21 D, 3720 Kortessem",NaN,False,Flat,219000€ 219.000,NaN,3,128m2,...,NaN,Yes,NaN,False,NaN,Hoogveldstraat 21 D,3720,Kortessem,3720,
18042,2,https://www.realo.be/en/eerste-meistraat-4-370...,"Eerste-Meistraat 4, 3700 Tongeren",NaN,False,Business,0,NaN,3,128m2,...,NaN,Yes,NaN,False,NaN,Eerste-Meistraat 4,3700,Tongeren,3700,
18043,2,https://www.realo.be/en/roeselaarsestraat-84-8...,"Roeselaarsestraat 84 8, 8870 Izegem",NaN,False,Flat,247000€ 247.000,NaN,2,125m2,...,NaN,Yes,NaN,False,NaN,Roeselaarsestraat 84 8,8870,Izegem,8870,
18044,2,https://www.realo.be/en/meikeverstraat-12-2250...,"Meikeverstraat 12, 2250 Olen",NaN,True,House,379000€ 379.000,NaN,4,220m2,...,NaN,Yes,NaN,False,NaN,Meikeverstraat 12,2250,Olen,2250,


In [246]:
#Garbage data to check out later
trash_value = "Zéér RUIME Bel-etage woning met ATELIER"
#trash_value = "Zug" WARNING CANNOT FILTER THESE, WHY?
df[df['locality'] == trash_value]

,source,hyperlink,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,...,open_fire,terrace,terrace_area,garden,garden_area,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state


In [226]:
df[df['merged_postcode'].isnull()]

,source,hyperlink,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,...,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state,extracted_address,extracted_postcode,extracted_locality,merged_postcode,unmerged_postcode
18034,2,https://www.realo.be/en/rue-rimiere-105-4120-r...,"Rue Rimière 105, 4120 Rotheux-Rimière, Neupré",NaN,True,House,0,NaN,0,0,...,NaN,No,NaN,False,NaN,Rue Rimière 105,4120,Rotheux-Rimière,NaN,NaN
18035,2,https://www.realo.be/en/sint-truidensesteenweg...,"Sint-Truidensesteenweg 95A A, 3350 Orsmaal-Gus...",NaN,True,House,340000€ 340.000,NaN,3,170m2,...,NaN,Yes,NaN,False,NaN,Sint-Truidensesteenweg 95A A,3350,Orsmaal-Gussenhoven,NaN,NaN
18036,2,https://www.realo.be/en/augustijnenlaan-77-220...,"Augustijnenlaan 77 201, 2200 Herentals",NaN,False,Flat,259000€ 259.000,NaN,2,88m2,...,NaN,Yes,NaN,False,NaN,Augustijnenlaan 77 201,2200,Herentals,NaN,NaN
18037,2,https://www.realo.be/en/averbodesesteenweg-19-...,"Averbodesesteenweg 19, 2230 Herselt",NaN,True,House,449000€ 449.000,NaN,4,303m2,...,NaN,Yes,NaN,False,NaN,Averbodesesteenweg 19,2230,Herselt,NaN,NaN
18038,2,https://www.realo.be/en/edward-pynaertkaai-21-...,"Edward pynaertkaai 21, 9050 Ledeberg",NaN,False,Flat,249900€ 249.900,NaN,3,102m2,...,NaN,Yes,NaN,False,NaN,Edward pynaertkaai 21,9050,Ledeberg,NaN,NaN
18039,2,https://www.realo.be/en/engelstraat-148-9040-s...,Zéér RUIME Bel-etage woning met ATELIER,NaN,True,House,495000€ 495.000,NaN,3,244m2,...,NaN,Yes,NaN,False,NaN,Zéér RUIME Bel-etage woning met ATELIER,Zéér RUIME Bel-etage woning met ATELIER,Zéér RUIME Bel-etage woning met ATELIER,NaN,NaN
18040,2,https://www.realo.be/en/groenplein-33-9060-zel...,"Groenplein 33 302, 9060 Zelzate",NaN,False,Flat,159000€ 159.000,NaN,2,244m2,...,NaN,Yes,NaN,False,NaN,Groenplein 33 302,9060,Zelzate,NaN,NaN
18041,2,https://www.realo.be/en/hoogveldstraat-21-3720...,"Hoogveldstraat 21 D, 3720 Kortessem",NaN,False,Flat,219000€ 219.000,NaN,3,128m2,...,NaN,Yes,NaN,False,NaN,Hoogveldstraat 21 D,3720,Kortessem,NaN,NaN
18042,2,https://www.realo.be/en/eerste-meistraat-4-370...,"Eerste-Meistraat 4, 3700 Tongeren",NaN,False,Business,0,NaN,3,128m2,...,NaN,Yes,NaN,False,NaN,Eerste-Meistraat 4,3700,Tongeren,NaN,NaN
18043,2,https://www.realo.be/en/roeselaarsestraat-84-8...,"Roeselaarsestraat 84 8, 8870 Izegem",NaN,False,Flat,247000€ 247.000,NaN,2,125m2,...,NaN,Yes,NaN,False,NaN,Roeselaarsestraat 84 8,8870,Izegem,NaN,NaN
